# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [41]:
import influxdb_client_3 as InfluxDBClient3
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, BatchNormalization, Normalization
from keras.models import Model
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed,  Conv1D, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from huggingface_hub import push_to_hub_keras
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
pio.renderers.default = "vscode"
import os, datetime


In [42]:
token = 'dzfx2CLrpmGjbTHmdbnw1cSB7Op89g6cQ2VD7EVRJYHiF8UN8bRqIaBK92TxnCa906qWjjV6KVk_TQrW0uH5IA=='
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'robot'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [43]:
query = '''SELECT MOVING_AVERAGE(last("coord"), 5) as coord FROM "joints" WHERE time >= 1697030174180ms and time <= 1697030462986ms GROUP BY time(3s) fill(linear)'''
table = client.query(query=query, language="influxql")
normal = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna(how='all')
fig = px.line(normal, y=["coord"], title='raw coord' )
fig.show()

In [44]:
query = '''SELECT MOVING_AVERAGE(last("coord"), 5) as coord FROM "joints" WHERE time >= 1697030663033ms and time <= 1697030872564ms GROUP BY time(3s) fill(linear)'''
table = client.query(query=query, language="influxql")
death_loop = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna(how='all')
fig = px.line(death_loop, y=["coord"], title='raw coord' )
fig.show()

In [45]:
query = '''SELECT MOVING_AVERAGE(last("coord"), 5) as coord FROM "joints" WHERE time >= 1697031873606ms and time <= 1697032040567ms GROUP BY time(3s) fill(linear)'''
table = client.query(query=query, language="influxql")
dropped = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna(how='all')
fig = px.line(dropped, y=["coord"], title='raw coord' )
fig.show()

In [46]:
# Function to create overlapping sequences
def create_overlapping_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

# Define the normalization layer
normalizer = Normalization(axis=-1)
normalizer.adapt(normal[['coord']].values)

sequence_length = 30  # Adjust this based on your preference
input_dim = 1  # Number of features

# Define the input layer
inputs = Input(shape=(sequence_length, input_dim))
normalized_inputs = normalizer(inputs)

# Define the LSTM layers
encoded = LSTM(16, activation='relu', return_sequences=True)(normalized_inputs)
encoded = LSTM(4, activation='relu', return_sequences=False)(encoded)
decoded = RepeatVector(sequence_length)(encoded)
decoded = LSTM(4, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(16, activation='relu', return_sequences=True)(decoded)
outputs = TimeDistributed(Dense(input_dim))(decoded)

autoencoder = Model(inputs, outputs)
autoencoder.compile(optimizer='adam', loss='mse')

# Prepare the data for training
X = normal[['coord']].values

# Create overlapping sequences
sequences = create_overlapping_sequences(X, sequence_length)

# Reshape the sequences for LSTM input [samples, timesteps, features]
sequences = sequences.reshape((sequences.shape[0], sequence_length, input_dim))

# Split the data into training and testing sets
X_train, X_test = train_test_split(sequences, test_size=0.2, random_state=42)

# Directory to store logs and TensorBoard callback
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=16, validation_data=(X_test, X_test), verbose=1, callbacks=[tensorboard_callback])

Epoch 1/100
4/4 [==============================] - 5s 262ms/step - loss: 6017.5132 - val_loss: 6400.4956
Epoch 2/100
4/4 [==============================] - 0s 64ms/step - loss: 6016.6748 - val_loss: 6399.6802
Epoch 3/100
4/4 [==============================] - 0s 67ms/step - loss: 6015.8892 - val_loss: 6398.8438
Epoch 4/100
4/4 [==============================] - 0s 64ms/step - loss: 6015.0757 - val_loss: 6397.9756
Epoch 5/100
4/4 [==============================] - 0s 62ms/step - loss: 6014.2163 - val_loss: 6397.0537
Epoch 6/100
4/4 [==============================] - 0s 61ms/step - loss: 6013.3374 - val_loss: 6396.0835
Epoch 7/100
4/4 [==============================] - 0s 71ms/step - loss: 6012.3662 - val_loss: 6395.0684
Epoch 8/100
4/4 [==============================] - 0s 64ms/step - loss: 6011.3750 - val_loss: 6393.9717
Epoch 9/100
4/4 [==============================] - 0s 66ms/step - loss: 6010.2959 - val_loss: 6392.7622
Epoch 10/100
4/4 [==============================] - 0s 64ms/ste

In [47]:
# Prepare the 'normal' data into sequences
sequence = create_overlapping_sequences(normal['coord'].values, sequence_length)
sequence = sequence.reshape((sequence.shape[0], sequence_length, input_dim))

# Use the Autoencoder to predict on the sequences
predictions = autoencoder.predict(sequence)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(sequence - predictions, 2), axis=(1, 2))

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 60  # Define a threshold value (in percentage)

# Handling the offset caused by sequence creation
offset = len(normal) - len(mse_percentage)

# Initialize columns with default values
normal['is_anomalous'] = False
normal['mse_percentage'] = np.nan
normal['threshold'] = threshold

# Use .loc to safely assign values
normal.loc[normal.index[offset:], 'is_anomalous'] = mse_percentage > threshold
normal.loc[normal.index[offset:], 'mse_percentage'] = mse_percentage




# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=normal.index,
    y=normal["coord"],
    mode='lines',
    name='coord',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=normal[normal['is_anomalous']].index,
    y=normal[normal['is_anomalous']]["coord"],
    mode='markers',
    name='Anomalous coord',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
normal.to_csv('normal_data_result.csv')

2/2 [==============================] - 1s 13ms/step


In [48]:
# Prepare the 'normal' data into sequences
sequences = create_overlapping_sequences(death_loop['coord'].values, sequence_length)
sequences = sequences.reshape((sequences.shape[0], sequence_length, input_dim))

# Use the Autoencoder to predict on the sequences
predictions = autoencoder.predict(sequences)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(sequences - predictions, 2), axis=(1, 2))

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 60  # Define a threshold value (in percentage)

# Handling the offset caused by sequence creation
offset = len(death_loop) - len(mse_percentage)

# Initialize columns with default values
death_loop['is_anomalous'] = False
death_loop['mse_percentage'] = np.nan
death_loop['threshold'] = threshold

# Use .loc to safely assign values
death_loop.loc[death_loop.index[offset:], 'is_anomalous'] = mse_percentage > threshold
death_loop.loc[death_loop.index[offset:], 'mse_percentage'] = mse_percentage



# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=death_loop.index,
    y=death_loop["coord"],
    mode='lines',
    name='coord',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=death_loop[death_loop['is_anomalous']].index,
    y=death_loop[death_loop['is_anomalous']]["coord"],
    mode='markers',
    name='Anomalous coord',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
death_loop.to_csv('death_loop_result.csv')

2/2 [==============================] - 0s 35ms/step


In [49]:
# Prepare the 'normal' data into sequences
sequences = create_overlapping_sequences(dropped['coord'].values, sequence_length)
sequences = sequences.reshape((sequences.shape[0], sequence_length, input_dim))

# Use the Autoencoder to predict on the sequences
predictions = autoencoder.predict(sequences)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(sequences - predictions, 2), axis=(1, 2))

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 60  # Define a threshold value (in percentage)

# Handling the offset caused by sequence creation
offset = len(dropped) - len(mse_percentage)

# Initialize columns with default values
dropped['is_anomalous'] = False
dropped['mse_percentage'] = np.nan
dropped['threshold'] = threshold

# Use .loc to safely assign values
dropped.loc[dropped.index[offset:], 'is_anomalous'] = mse_percentage > threshold
dropped.loc[dropped.index[offset:], 'mse_percentage'] = mse_percentage



# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=dropped.index,
    y=dropped["coord"],
    mode='lines',
    name='coord',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=dropped[dropped['is_anomalous']].index,
    y=dropped[dropped['is_anomalous']]["coord"],
    mode='markers',
    name='Anomalous coord',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
dropped.to_csv('dropped_result.csv')

1/1 [==============================] - 0s 30ms/step
